# Plotting with Matplotlib

Python has great libraries for scientific plotting, the most important of which is [`matplotlib`](https://matplotlib.org/). Let's use it to examine voting patterns in Stavanger!

## Getting some data

In [ ]:
import csv
from io import StringIO
from itertools import chain, islice
from urllib.request import urlopen
import matplotlib.pyplot as plt
import numpy

url = 'https://opencom.no/dataset/d93c7128-2e18-4513-9724-7a60d49b206c/resource/5a73e65e-140c-4137-bbc5-6decebaac879/download/ssbdata.csv'

# Get data shaped like:
# { region-name: {party-name: percent } }

with urlopen(url) as handle:
    csv_data = StringIO(handle.read().decode('utf-8'))
    csv_data.seek(0)
    reader = csv.reader(csv_data, delimiter=',')
    data = {}
    for region, party, percent in islice(reader, 1, None):
        try:
            data.setdefault(region, {})[party] = float(percent)
        except ValueError:
            # Probably just the "." null-values markers in the percent data
            pass
        
all_parties = sorted(set(chain(*(rset for rset in data.values()))))

## How did people in my neighborhood vote?

In [ ]:
fig, ax = plt.subplots(dpi=150)

kvaleberg = data['Kvaleberg']
parties, percents = tuple(zip(*kvaleberg.items()))
_ = ax.pie(percents, labels=parties)

## How did the islands vote?

### By party

In [ ]:
islands = tuple(region for region in data if 'øy' in region.lower())

island_data = [
    (region, [rset.get(party, 0) 
              for party 
              in all_parties])
    for region, rset
    in data.items()
    if region in islands
]

x = numpy.array(range(len(all_parties)), dtype='float64')
width = 0.2

fig, ax = plt.subplots(dpi=150)
for index, (region, percents) in enumerate(island_data):
    ax.bar(x + index * width, percents, width=width, label=region)
    
ax.set_xticks(range(len(all_parties)))
ax.set_xticklabels(all_parties)
ax.tick_params(axis='x', labelrotation=90)
ax.legend()
plt.gcf().subplots_adjust(bottom=0.45)

### By island

In [ ]:
x = numpy.array(range(len(islands)), dtype='float64')
width = 0.08

fig, ax = plt.subplots(dpi=150)

for index, party in enumerate(all_parties):
    percs = [data[region].get(party, 0.0) for region in islands]
    ax.bar(x + index * width, percs, width=width, label=party)

ax.set_xticks(range(len(islands)))
ax.set_xticklabels(islands)
ax.legend(loc='center left', bbox_to_anchor=(0.83,0.8), prop={'size': 6})
plt.show()